<a href="https://colab.research.google.com/github/Banafshehkh/Fake_News_Detection/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Author: Banafsheh Khazali
#Date: April 27, 2023

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [49]:
import numpy as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer #Convert a collection of text documents to a matrix of token counts.
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

## **Load the Data**

In [50]:
train_df = pd.read_csv('/content/drive/MyDrive/Data/train_preprocessed.csv')
val_df = pd.read_csv('/content/drive/MyDrive/Data/valid_preprocessed.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Data/test_preprocessed.csv')

In [51]:
test_df

,Statement,Label
0,build wall u s mexico border take liter year,True
1,wisconsin pace doubl number layoff year,False
2,say john mccain done noth help vet,False
3,suzann bonamici support plan cut choic medicar...,True
4,ask report whether he center crimin scheme vio...,False
...,...,...
2546,first time decad import account less half the ...,True
2547,say donald trump bankrupt compani onc twice fo...,True
2548,john mccain georg bush absolut plan univers he...,True
2549,new poll show 62 percent support presid s plan...,False


In [70]:
val_df.Label = val_df['Label'].str.lower()


In [71]:
train_df.Label =  train_df['Label'].replace({True: 1, False: 0})
val_df.Label = val_df['Label'].replace({True: 1, False: 0})
test_df.Label = test_df['Label'].replace({True: 1, False: 0})


In [74]:
print(val_df["Label"].unique())

['false' 'true' 'label']


In [ ]:
label_rows = val_df['Label'] == 'label'

In [62]:
vectorizer = CountVectorizer(stop_words='english')

X_train = vectorizer.fit_transform(train_df['Statement'])
y_train = train_df['Label']
y_train = y_train.astype(int)

X_val = vectorizer.transform(val_df['Statement'])
y_val = val_df['Label']
print(y_val.dtype)
# y_val = y_val.astype(int)

X_test = vectorizer.transform(test_df['Statement'])
y_test = test_df['Label']
y_test = y_test.astype(int)


object


In [47]:
y_val.dtype

dtype('O')

## **Train a Machine Learning Model**

naive bayes

In [34]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

ValueError: ignored

## **Evaluate the Model**

In [28]:
y_pred_val = clf.predict(X_val)
y_pred_val.dtype


dtype('int64')

In [30]:
y_val.dtype

dtype('O')

In [31]:
y_val = y_val.astype(int)


ValueError: ignored

In [26]:
val_accuracy = accuracy_score(y_val, y_pred_val)
print(f"Validation accuracy: {val_accuracy:.2f}")

Validation accuracy: 0.00


## **Test the Model**